In [1]:
mamba activate chrombpnet2

In [2]:
#export PATH=/cluster/home/jjanssens/.local/bin:$PATH

In [3]:
export LD_LIBRARY_PATH=/cluster/project/treutlein/jjans/software/miniforge3/envs/cuda11_env/lib:$CONDA_PREFIX/lib:$LD_LIBRARY_PATH

In [4]:
echo $LD_LIBRARY_PATH

/cluster/project/treutlein/jjans/software/miniforge3/envs/cuda11_env/lib:/cluster/project/treutlein/jjans/software/miniforge3/envs/chrombpnet2/lib:/cluster/software/manual/julia/1.10.3/x86_64/lib:/cluster/software/stacks/2024-05/spack/opt/spack/linux-ubuntu22.04-x86_64_v3/gcc-13.2.0/r-4.3.2-c6hxdn6t4ao6fbqv4cg2xqinhwtwdwbn/rlib:/cluster/software/stacks/2024-05/spack/opt/spack/linux-ubuntu22.04-x86_64_v3/gcc-13.2.0/r-4.3.2-c6hxdn6t4ao6fbqv4cg2xqinhwtwdwbn/rlib/R/lib:/cluster/software/stacks/2024-05/spack/opt/spack/linux-ubuntu22.04-x86_64_v3/gcc-13.2.0/cuda-12.2.1-fv6yilrnidoe35nh24iszjkf63bihikt/lib64:/cluster/software/stacks/2024-05/spack/opt/spack/linux-ubuntu22.04-x86_64_v3/gcc-13.2.0/python-3.11.6-m4n2ny4dwaqmeobuqquo3gpyw2bct67i/lib:/cluster/software/stacks/2024-05/spack/opt/spack/linux-ubuntu22.04-x86_64_v3/gcc-11.4.0/gcc-13.2.0-6t2bdhlkrktlyopht3nbmjz3emlsqeog/lib64:/cluster/software/stacks/2024-05/spack/opt/spack/linux-ubuntu22.04-x86_64_v3/gcc-11.4.0/gcc-13.2.0-6t2bdhlkrktlyop

In [5]:
which nvcc

/cluster/software/stacks/2024-05/spack/opt/spack/linux-ubuntu22.04-x86_64_v3/gcc-13.2.0/cuda-12.2.1-fv6yilrnidoe35nh24iszjkf63bihikt/bin/nvcc


In [6]:
which chrombpnet

/cluster/project/treutlein/jjans/software/miniforge3/envs/chrombpnet2/bin/chrombpnet


In [7]:
which python

/cluster/project/treutlein/jjans/software/miniforge3/envs/chrombpnet2/bin/python


In [8]:
unset PYTHONPATH

In [9]:
which python

/cluster/project/treutlein/jjans/software/miniforge3/envs/chrombpnet2/bin/python


In [10]:
for gene in "LCT" "AIG" "SLC5A12" "IGFBP2"
do
    bedtools intersect -a celltype_models_human/called_peaks/enterocytes_peaks_no_blacklist_chr.bed -b regions_of_interest/${gene}_regions.bed -wa > regions_of_interest/${gene}_regions_peaks_human.bed
done

In [11]:
head regions_of_interest/LCT_regions_peaks_human.bed

chr2	135850768	135851620	human_peaks_peak_46168	428	.	9.08804	45.6806	42.8412	282


In [13]:
ls celltype_models_human/modelv1_EEC/fold_4

output	output_auxiliary  output_negatives.bed


In [15]:
# Define input variables
cell_types=("enterocytes" "Stem_cell" "Goblet_cell" "EEC")
folds=("fold_0" "fold_1" "fold_2" "fold_3" "fold_4")
genes=("IGFBP2" "LCT" "AIG" "SLC5A12")

# Create a log file for failed jobs
failed_log="failed_jobs.log"
echo "Failed Jobs:" > "$failed_log"

# Function to run chrombpnet contribs_bw and handle errors
run_contribs() {
    local cell_type="$1"
    local fold="$2"
    local region_name="$3"
    local log_file="$4"
    
    local model_path="celltype_models_human/modelv1_${cell_type}/${fold}/output/models/chrombpnet_nobias.h5"
    local regions="regions_of_interest/${region_name}_regions_peaks_human.bed"
    local genome_fasta="encode_data/hg38.fa"
    local genome_sizes="encode_data/hg38.chrom.sizes"
    local output_prefix="region_scores/${cell_type}/${region_name}_regions/${fold}"
    
    mkdir -p "region_scores/${cell_type}/${region_name}_regions"
        
    # Run chrombpnet contribs_bw command
    chrombpnet contribs_bw \
        -m "$model_path" \
        -r "$regions" \
        -g "$genome_fasta" \
        -c "$genome_sizes" \
        -op "$output_prefix" &> "${output_prefix}.log"
    
    # Capture the exit status of the command
    exit_status=$?
    
    # Check if the command failed
    if [ $exit_status -ne 0 ]; then
        echo "Job failed for cell_type: $cell_type, fold: $fold, region_name: $region_name" >> "$log_file"
    fi
}


# Export the function so that GNU Parallel can access it
export -f run_contribs

# Use GNU Parallel to iterate over combinations of cell types, folds, and genes
parallel -j 1 run_contribs {1} {2} {3} "$failed_log" ::: "${cell_types[@]}" ::: "${folds[@]}" ::: "${genes[@]}"
